In [12]:
pip install requests

Note: you may need to restart the kernel to use updated packages.Collecting requests
     ---------------------------------------- 62.8/62.8 kB 3.3 MB/s eta 0:00:00
     ---------------------------------------- 96.0/96.0 kB 5.7 MB/s eta 0:00:00
     ---------------------------------------- 61.5/61.5 kB ? eta 0:00:00
     -------------------------------------- 140.6/140.6 kB 8.2 MB/s eta 0:00:00
     -------------------------------------- 155.3/155.3 kB 9.7 MB/s eta 0:00:00



  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip available: 22.3.1 -> 23.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [14]:
pip install pandas

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip available: 22.3.1 -> 23.0
[notice] To update, run: python.exe -m pip install --upgrade pip



     --------------------------------------- 10.3/10.3 MB 13.9 MB/s eta 0:00:00
     ------------------------------------- 499.4/499.4 kB 15.8 MB/s eta 0:00:00
     --------------------------------------- 14.8/14.8 MB 14.2 MB/s eta 0:00:00


In [36]:
import requests
import json
import time
import pandas as pd
from pprint import pprint


# 1. 楽天レシピのレシピカテゴリ一覧を取得する

res = requests.get('https://app.rakuten.co.jp/services/api/Recipe/CategoryList/20170426?applicationId=1050436635711957152')

json_data = json.loads(res.text)

parent_dict = {} # mediumカテゴリの親カテゴリの辞書

df = pd.DataFrame(columns=['category1','category2','category3','categoryId','categoryName'])

for category in json_data['result']['large']:
    df = pd.concat([df, pd.DataFrame({'category1':category['categoryId'],'category2':"",'category3':"",'categoryId':category['categoryId'],'categoryName':category['categoryName']}, index=[0])], ignore_index=True)

for category in json_data['result']['medium']:
    df = pd.concat([df, pd.DataFrame({'category1':category['parentCategoryId'],'category2':category['categoryId'],'category3':"",'categoryId':str(category['parentCategoryId'])+"-"+str(category['categoryId']),'categoryName':category['categoryName']}, index=[0])], ignore_index=True)
    parent_dict[str(category['categoryId'])] = category['parentCategoryId']

for category in json_data['result']['small']:
    df = pd.concat([df,pd.DataFrame({'category1':parent_dict[category['parentCategoryId']],'category2':category['parentCategoryId'],'category3':category['categoryId'],'categoryId':parent_dict[category['parentCategoryId']]+"-"+str(category['parentCategoryId'])+"-"+str(category['categoryId']),'categoryName':category['categoryName']}, index=[0])], ignore_index=True)


pprint(json_data)

{'result': {'large': [{'categoryId': '30',
                       'categoryName': '人気メニュー',
                       'categoryUrl': 'https://recipe.rakuten.co.jp/category/30/'},
                      {'categoryId': '31',
                       'categoryName': '定番の肉料理',
                       'categoryUrl': 'https://recipe.rakuten.co.jp/category/31/'},
                      {'categoryId': '32',
                       'categoryName': '定番の魚料理',
                       'categoryUrl': 'https://recipe.rakuten.co.jp/category/32/'},
                      {'categoryId': '33',
                       'categoryName': '卵料理',
                       'categoryUrl': 'https://recipe.rakuten.co.jp/category/33/'},
                      {'categoryId': '14',
                       'categoryName': 'ご飯もの',
                       'categoryUrl': 'https://recipe.rakuten.co.jp/category/14/'},
                      {'categoryId': '15',
                       'categoryName': 'パスタ',
                       'categoryUrl'

In [65]:
import requests
import json
import time
import pandas as pd
from pprint import pprint


# 1. 楽天レシピのレシピカテゴリ一覧を取得する

res = requests.get('https://app.rakuten.co.jp/services/api/Recipe/CategoryList/20170426?applicationId=1050436635711957152')

json_data = json.loads(res.text)

parent_dict = {} # mediumカテゴリの親カテゴリの辞書

df = pd.DataFrame(columns=['category1','category2','category3','categoryId','categoryName'])

for category in json_data['result']['large']:
    df = pd.concat([df, pd.DataFrame({'category1':category['categoryId'],'category2':"",'category3':"",'categoryId':category['categoryId'],'categoryName':category['categoryName']}, index=[0])], ignore_index=True)

for category in json_data['result']['medium']:
    df = pd.concat([df, pd.DataFrame({'category1':category['parentCategoryId'],'category2':category['categoryId'],'category3':"",'categoryId':str(category['parentCategoryId'])+"-"+str(category['categoryId']),'categoryName':category['categoryName']}, index=[0])], ignore_index=True)
    parent_dict[str(category['categoryId'])] = category['parentCategoryId']

for category in json_data['result']['small']:
    df = pd.concat([df,pd.DataFrame({'category1':parent_dict[category['parentCategoryId']],'category2':category['parentCategoryId'],'category3':category['categoryId'],'categoryId':parent_dict[category['parentCategoryId']]+"-"+str(category['parentCategoryId'])+"-"+str(category['categoryId']),'categoryName':category['categoryName']}, index=[0])], ignore_index=True)

# キーワードを含む行を抽出
df_keyword = df.query('categoryName.str.contains("肉")', engine='python')

pprint(df_keyword)


     category1 category2 category3   categoryId   categoryName
1           31                               31         定番の肉料理
12          10                               10              肉
43          10       275                 10-275             牛肉
44          10       276                 10-276             豚肉
45          10       277                 10-277             鶏肉
...        ...       ...       ...          ...            ...
1851        40       521      1742  40-521-1742  ホーロー鍋で作る肉のおかず
1875        41       537      1766  41-537-1766    ホイコーロー（回鍋肉）
1885        41       547      1776  41-547-1776            肉まん
2059        19       675      1569  19-675-1569            肉味噌
2155        31       719      2129  31-719-2129            肉巻き

[77 rows x 5 columns]


In [72]:
import requests
import json
import time
import pandas as pd
from pprint import pprint


# 1. 楽天レシピのレシピカテゴリ一覧を取得する

res = requests.get('https://app.rakuten.co.jp/services/api/Recipe/CategoryList/20170426?applicationId=1050436635711957152')

json_data = json.loads(res.text)

parent_dict = {} # mediumカテゴリの親カテゴリの辞書

df = pd.DataFrame(columns=['category1','category2','category3','categoryId','categoryName'])

for category in json_data['result']['large']:
    df = pd.concat([df, pd.DataFrame({'category1':category['categoryId'],'category2':"",'category3':"",'categoryId':category['categoryId'],'categoryName':category['categoryName']}, index=[0])], ignore_index=True)

for category in json_data['result']['medium']:
    df = pd.concat([df, pd.DataFrame({'category1':category['parentCategoryId'],'category2':category['categoryId'],'category3':"",'categoryId':str(category['parentCategoryId'])+"-"+str(category['categoryId']),'categoryName':category['categoryName']}, index=[0])], ignore_index=True)
    parent_dict[str(category['categoryId'])] = category['parentCategoryId']

for category in json_data['result']['small']:
    df = pd.concat([df,pd.DataFrame({'category1':parent_dict[category['parentCategoryId']],'category2':category['parentCategoryId'],'category3':category['categoryId'],'categoryId':parent_dict[category['parentCategoryId']]+"-"+str(category['parentCategoryId'])+"-"+str(category['categoryId']),'categoryName':category['categoryName']}, index=[0])], ignore_index=True)

# キーワードを含む行を抽出
df_keyword = df.query('categoryName.str.contains("肉")', engine='python')

# 人気レシピを取得
res = requests.get('https://app.rakuten.co.jp/services/api/Recipe/CategoryRanking/20170426?applicationId=1050436635711957152')

df_recipe = pd.DataFrame(columns=['recipeId', 'recipeTitle', 'foodImageUrl',  'categoryId', 'categoryName'])

#for index, row in df_keyword.iterrows():
#    time.sleep(3) # 連続でアクセスすると先方のサーバに負荷がかかるので少し待つのがマナー
#
#    url = 'https://app.rakuten.co.jp/services/api/Recipe/CategoryRanking/20170426?applicationId=1050436635711957152&categoryId='+row['categoryId']
#    res = requests.get(url)
#
#    json_data = json.loads(res.text)
#    recipes = json_data['result']

json_data = json.loads(res.text)

pprint(json_data)

{'result': [{'foodImageUrl': 'https://image.space.rakuten.co.jp/d/strg/ctrl/3/d61acc38e689e03689f0e1641b19d925fcb49f55.44.2.3.2.jpg',
             'mediumImageUrl': 'https://image.space.rakuten.co.jp/d/strg/ctrl/3/d61acc38e689e03689f0e1641b19d925fcb49f55.44.2.3.2.jpg?thum=54',
             'nickname': 'にゃんぺこ',
             'pickup': 0,
             'rank': '1',
             'recipeCost': '100円以下',
             'recipeDescription': '白菜だけなのにバンバンご飯がすすんじゃう( ´ ▽ ` '
                                  ')ﾉ難しいことも一切無しの簡単レシピ♪丼の餡としても最高の味です(｡•̀ᴗ-)و '
                                  '̑̑✧',
             'recipeId': 1000017354,
             'recipeIndication': '約10分',
             'recipeMaterial': ['白菜',
                                '★酒/オイスターソース',
                                '★醤油/みりん/豆板醤',
                                '★鶏がらスープの素',
                                '★生姜(チューブ)',
                                '☆片栗粉',
                                '☆水',
                                '黒胡椒'

In [104]:
import requests
import json
import time
import pandas as pd
from pprint import pprint


# 1. 楽天レシピのレシピカテゴリ一覧を取得する

res = requests.get('https://app.rakuten.co.jp/services/api/Recipe/CategoryList/20170426?applicationId=1050436635711957152')

json_data = json.loads(res.text)

parent_dict = {} # mediumカテゴリの親カテゴリの辞書

df = pd.DataFrame(columns=['category1','category2','category3','categoryId','categoryName'])

for category in json_data['result']['large']:
    df = pd.concat([df, pd.DataFrame({'category1':category['categoryId'],'category2':"",'category3':"",'categoryId':category['categoryId'],'categoryName':category['categoryName']}, index=[0])], ignore_index=True)

for category in json_data['result']['medium']:
    df = pd.concat([df, pd.DataFrame({'category1':category['parentCategoryId'],'category2':category['categoryId'],'category3':"",'categoryId':str(category['parentCategoryId'])+"-"+str(category['categoryId']),'categoryName':category['categoryName']}, index=[0])], ignore_index=True)
    parent_dict[str(category['categoryId'])] = category['parentCategoryId']

for category in json_data['result']['small']:
    df = pd.concat([df,pd.DataFrame({'category1':parent_dict[category['parentCategoryId']],'category2':category['parentCategoryId'],'category3':category['categoryId'],'categoryId':parent_dict[category['parentCategoryId']]+"-"+str(category['parentCategoryId'])+"-"+str(category['categoryId']),'categoryName':category['categoryName']}, index=[0])], ignore_index=True)


df_keyword = df.query('categoryName.str.contains("魚")', engine='python')

# 人気レシピを取得
res = requests.get('https://app.rakuten.co.jp/services/api/Recipe/CategoryRanking/20170426?applicationId=1050436635711957152')
json_data = json.loads(res.text)
df_recipe = pd.DataFrame(columns=['recipeId', 'recipeTitle', 'foodImageUrl', 'recipeMaterial', 'recipeCost', 'recipeIndication', 'categoryId', 'categoryName'])

for index, row in df_recipe.iterrows():
    time.sleep(3) # 連続でアクセスすると先方のサーバに負荷がかかるので少し待つのがマナー

    url = 'https://app.rakuten.co.jp/services/api/Recipe/CategoryRanking/20170426?applicationId=1050436635711957152'
    res = requests.get(url)

    json_data = json.loads(res.text)
    recipes = json_data['result']

    for recipe in recipes:
        df_recipe = df_recipe.concat([df,pd.DataFrame({'recipeId':recipe['recipeId'],'recipeTitle':recipe['recipeTitle'],'foodImageUrl':recipe['foodImageUrl'],'recipeMaterial':recipe['recipeMaterial'],'recipeCost':recipe['recipeCost'],'recipeIndication':recipe['recipeIndication'],'categoryId':row['categoryId'],'categoryName':row['categoryName']}, index=[0])], ignore_index=True)

pprint(df_recipe)

Empty DataFrame
Columns: [recipeId, recipeTitle, foodImageUrl, recipeMaterial, recipeCost, recipeIndication, categoryId, categoryName]
Index: []


In [108]:
import requests
import json
import time
import pandas as pd
from pprint import pprint

# 1. 楽天レシピのレシピカテゴリ一覧を取得する

res = requests.get('https://app.rakuten.co.jp/services/api/Recipe/CategoryList/20170426?applicationId=1050436635711957152')

json_data = json.loads(res.text)

parent_dict = {} # mediumカテゴリの親カテゴリの辞書

df = pd.DataFrame(columns=['category1','category2','category3','categoryId','categoryName'])

for category in json_data['result']['large']:
    df = pd.concat([df, pd.DataFrame({'category1':category['categoryId'],'category2':"",'category3':"",'categoryId':category['categoryId'],'categoryName':category['categoryName']}, index=[0])], ignore_index=True)

for category in json_data['result']['medium']:
    df = pd.concat([df, pd.DataFrame({'category1':category['parentCategoryId'],'category2':category['categoryId'],'category3':"",'categoryId':str(category['parentCategoryId'])+"-"+str(category['categoryId']),'categoryName':category['categoryName']}, index=[0])], ignore_index=True)
    parent_dict[str(category['categoryId'])] = category['parentCategoryId']

for category in json_data['result']['small']:
    df = pd.concat([df,pd.DataFrame({'category1':parent_dict[category['parentCategoryId']],'category2':category['parentCategoryId'],'category3':category['categoryId'],'categoryId':parent_dict[category['parentCategoryId']]+"-"+str(category['parentCategoryId'])+"-"+str(category['categoryId']),'categoryName':category['categoryName']}, index=[0])], ignore_index=True)

# 2. キーワードからカテゴリを抽出する
df_keyword = df.query('categoryName.str.contains("魚")', engine='python')

# 3. 人気レシピを取得する
df_recipe = pd.DataFrame(columns=['recipeId', 'recipeTitle', 'foodImageUrl', 'recipeMaterial', 'recipeCost', 'recipeIndication', 'categoryId', 'categoryName'])

for index, row in df_keyword.iterrows():
    time.sleep(3) # 連続でアクセスすると先方のサーバに負荷がかかるので少し待つのがマナー

    url = 'https://app.rakuten.co.jp/services/api/Recipe/CategoryRanking/20170426?applicationId=1050436635711957152&categoryId='+row['categoryId']
    res = requests.get(url)

    json_data = json.loads(res.text)
    recipes = json_data['result']

    for recipe in recipes:
        df_recipe = pd.concat([df,pd.DataFrame({'recipeId':recipe['recipeId'],'recipeTitle':recipe['recipeTitle'],'foodImageUrl':recipe['foodImageUrl'],'recipeMaterial':recipe['recipeMaterial'],'recipeCost':recipe['recipeCost'],'recipeIndication':recipe['recipeIndication'],'categoryId':row['categoryId'],'categoryName':row['categoryName']}, index=[0])], ignore_index=True)


ValueError: Length of values (7) does not match length of index (1)